In [19]:
import pandas as pd
import numpy as np

In [20]:
space_units_df = pd.read_csv('../Data/space_units_df.csv', index_col='Unit_Name')
ground_units_df= pd.read_csv('../Data/ground_units_df.csv', index_col='Unit_Name')

In [21]:
# Split Units into Ships and Ground Forces

def split_units(units):
    ships = {}
    ground_forces = {}

    for unit, value in units.items():
        if unit in space_units_df.index:
            ships[unit] = value
        elif unit in ground_units_df.index:
            ground_forces[unit] = value
            
    return ships, ground_forces

In [22]:
# Find Combat Values

def get_unit_stats(faction_units, units_df):
    faction_unit_stats = []
    for unit in faction_units:
        number_units = faction_units[unit]
        for i in range(number_units):
            name = unit
            unit_combat_value = int(units_df.loc[unit]['Combat_Value'])
            has_sustain_damage = units_df.loc[unit]['Has_Sustain_Damage']
            shots = units_df.loc[unit]['Shots']
            unit_stats = {'Name': name, 'Unit_Combat_Value': unit_combat_value, 'Shots': shots, 'Has_Sustain_Damage': has_sustain_damage}
            faction_unit_stats.append(unit_stats)
    faction_unit_stats.sort(key = lambda x: (x['Has_Sustain_Damage'], x['Unit_Combat_Value'], -x['Shots']), reverse=True)
    return faction_unit_stats

def get_ship_stats(faction_units):
    faction_unit_stats = get_unit_stats(faction_units, space_units_df)
    return faction_unit_stats

def get_ground_force_stats(faction_units):
    faction_unit_stats = get_unit_stats(faction_units, ground_units_df)
    return faction_unit_stats

In [23]:
# Determine Hits

def get_hits(faction_units):
    hits = 0
    for unit in faction_units:
        for i in range(unit['Shots']):
            roll = np.random.randint(1, 11)
            if roll >= unit['Unit_Combat_Value']:
                hits += 1
    return hits

In [24]:
# Assign Hits

def assign_hits(hits, faction_units):
    while hits > 0 and faction_units:
        if faction_units[0]['Has_Sustain_Damage']:
            faction_units[0]['Has_Sustain_Damage'] = False
            faction_units.sort(key = lambda x: (x['Has_Sustain_Damage'], x['Unit_Combat_Value'], -x['Shots']), reverse=True)
        else:
            faction_units.remove(faction_units[0])
        hits -= 1

In [35]:
# MAIN

# For this purpose faction_A is the attacker and faction_B is the defender
# If faction_B wins there is no ground combat.

# To Do:
# Account for anti-fighter barrage


faction_A_space_wins, faction_B_space_wins, space_draws = 0, 0, 0
faction_A_ground_wins, faction_B_ground_wins, ground_draws = 0, 0, 0
space_round_counts = []
ground_round_counts = []

for i in range(0, 10000):

    faction_A_units = {'Carrier': 1, 'Infantry': 1}
    faction_B_units = {'Dreadnought': 4, 'Infantry': 2}

    # Split Ground and Space Units
    faction_A_ships, faction_A_ground_forces = split_units(faction_A_units)
    faction_B_ships, faction_B_ground_forces = split_units(faction_B_units)

    # Find Combat Values
    faction_A_ships = get_ship_stats(faction_A_ships)
    faction_B_ships = get_ship_stats(faction_B_ships)
    faction_A_ground_forces = get_ground_force_stats(faction_A_ground_forces)
    faction_B_ground_forces = get_ground_force_stats(faction_B_ground_forces)

    space_round_count = 0
    while faction_A_ships and faction_B_ships:
        # Determine Hits
        faction_A_hits = get_hits(faction_A_ships)
        faction_B_hits = get_hits(faction_B_ships)

        # Assign Hits
        assign_hits(faction_A_hits, faction_B_ships)
        assign_hits(faction_B_hits, faction_A_ships)

        space_round_count += 1
    
    space_round_counts.append(space_round_count)

    if faction_A_ships:
        faction_A_space_wins += 1

        ground_round_count = 0
        while faction_A_ground_forces and faction_B_ground_forces:
            # Determine Hits
            faction_A_hits = get_hits(faction_A_ground_forces)
            faction_B_hits = get_hits(faction_B_ground_forces)

            # Assign Hits
            assign_hits(faction_A_hits, faction_B_ground_forces)
            assign_hits(faction_B_hits, faction_A_ground_forces)

            ground_round_count += 1
        
        ground_round_counts.append(ground_round_count)

        if faction_A_ground_forces:
            faction_A_ground_wins += 1
        elif faction_B_ground_forces:
            faction_B_ground_wins += 1
        else:
            ground_draws += 1

    elif faction_B_ships:
        faction_B_space_wins += 1
    else:
        space_draws += 1

average_num_space_rounds = sum(space_round_counts) / len(space_round_counts)
print(f'Average Number of Space Rounds: {average_num_space_rounds:.2f}')

average_num_ground_rounds = sum(ground_round_counts) / len(ground_round_counts) if ground_round_counts else 0
print(f'Average Number of Ground Rounds: {average_num_ground_rounds:.2f}')

total_space_games = faction_A_space_wins + faction_B_space_wins + space_draws
print(f'Number of Space Games Ran: {total_space_games}')

total_ground_games = faction_A_ground_wins + faction_B_ground_wins + ground_draws
print(f'Number of Ground Games Ran: {total_ground_games}')

df = pd.DataFrame([{'Faction A wins': faction_A_space_wins, 'Faction B wins': faction_B_space_wins, 'Draw': space_draws},
                   {'Faction A wins': faction_A_ground_wins, 'Faction B wins': faction_B_ground_wins, 'Draw': ground_draws}],
                   index=['Space Percentages', 'Ground Percentages'])

divisors = pd.Series([total_space_games, total_ground_games if total_ground_games > 0 else 1], index=df.index)
df = df.div(divisors, axis=0)

faction_A_wins_both = df.loc['Space Percentages', 'Faction A wins'] * df.loc['Ground Percentages', 'Faction A wins']
faction_B_wins_either = df.loc['Space Percentages', 'Faction B wins'] + (df.loc['Space Percentages', 'Faction A wins'] * df.loc['Ground Percentages', 'Faction B wins'])
draw_either = df.loc['Space Percentages', 'Draw'] + df.loc['Ground Percentages', 'Draw']
df.loc['Overall Percentages'] = {'Faction A wins': faction_A_wins_both, 'Faction B wins': faction_B_wins_either, 'Draw': draw_either}
df = df * 100

df.round(1)

Average Number of Space Rounds: 1.03
Average Number of Ground Rounds: 0.00
Number of Space Games Ran: 10000
Number of Ground Games Ran: 0


,Faction A wins,Faction B wins,Draw
Space Percentages,0.0,100.0,0.0
Ground Percentages,0.0,0.0,0.0
Overall Percentages,0.0,100.0,0.0
